In [1]:
from numpy import array

## Grammar
/!\ Ne pas retranscrire en Ocaml, il s'agit d'une version python de la grammaire d'Arbogen.

In [2]:
class Elem(str):
    
    def __init__(self, string):
        self = string
        
    def __str__(self):
        return self

In [3]:
class Component():
    
    def __init__(self, string, rule_name):
        string = string.replace(' ','').split('*')
        self.int = 0
        self.elements = []
        for s in string:
            if s[:2] == '<z':
                if s[2] == '>':
                    self.int += 1
                else:
                    self.int += int(s[3])
            if s == rule_name:
                self.elements.append(Elem(s))     
                
    def __str__(self):
        if self.int == 0:
            string = "Eps * "
        elif self.int == 1:
            string = "<z> * "
        else:
            string = "<z^" + str(self.int) + "> * "
        for element in self.elements:
            string += str(element) + " * "
        return string[:-3]

In [4]:
class Rule:
    
    def __init__(self, string):
        self.name,string = string.replace(' ','').split('=')
        self.components = []
        for s in string.split('+'):
            self.components.append(Component(s,self.name))
    
    def __str__(self):
        string = self.name + ' = '
        for component in self.components:
            string += str(component) + " + "
        return string[:-3]

In [5]:
R = Rule("B = <z^0> + <z> * B + <z^2> * B * B")
print(R)

B = Eps + <z> * B + <z^2> * B * B


## Utils

In [6]:
def number_of_recursions(component):
    return len(component.elements)

def number_of_nodes(component):
    return component.int

def number_of_leafs(component):
    if component.int == 0:
        return 1
    else:
        return 0

In [7]:
def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

In [8]:
def next_partition_with_factor(base):
    modification = False
    duplicate = 1
    den = 1
    for i in range(0, len(base) - 1):
        if not modification and base[i] - 1 >= base[i + 1] + 1:
            base[i] -= 1
            base[i + 1] += 1
            modification = True
        if base[i] == base[i+1]:
            duplicate += 1
            den *= duplicate
        else:
            duplicate = 1
    return modification,factorial(len(base))//den, base
                

    

## Backup

In [13]:
def add_in_backup(backup,number_of_element,e):
    if backup.size != number_of_element:
        backup[number_of_element] = e
    else:
        backup_increased = array([0]*2*number_of_element)
        for i in range(number_of_element):
            backup_increased[i] = backup[i]
            backup = backup_increased
    return backup

## Count

In [20]:
def count(N,rule,verbose = False):
    
    iFile = open("backup.cnt", 'r')
    string = iFile.read().split('\n')
    iFile.close()
    
    backup = array([0] * max(len(string)*2,1000))
    number_of_elements = 0
    for i in range(len(string) - 1):
        backup[i] = string[i]
        number_of_elements += 1
    
    oFile = open("backup.cnt",'a')
    for n in range(number_of_elements,N + 1):
        Bn = 0
        for component in rule.components:
            
            if n - number_of_nodes(component) >= 0:
                if number_of_recursions(component) == 0 and number_of_nodes(component) == n:
                    Bn += 1
                else:
                    base = [n - number_of_nodes(component)] + [0]*(number_of_recursions(component) - 1)
                    modification = True
                    if number_of_nodes(component) == n:
                        factor = 1
                    else: factor = number_of_recursions(component)
                    while modification:
                        product = 1
                        for e in base:
                            product *= backup[e]
                        Bn += product * factor
                        modification, factor, base =  next_partition_with_factor(base)
                        
        if backup.size == number_of_elements:
            backup_increased = array([0]*2*number_of_elements)
            for i in range(number_of_elements):
                backup_increased[i] = backup[i]
                backup = backup_increased
        backup[number_of_elements] = Bn
        number_of_elements += 1
        
    return backup[:number_of_elements]

In [21]:
R = Rule("B = <z^0> + <z^1> + <z> * B + <z> * B * B")
count(5,R)

array([  1,   3,   9,  36, 162, 783])